In [1]:
import sys
import os
# Get the parent directory path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

Load the images names

In [2]:
images = []
dataDir = "../images1/"
for img in os.listdir(dataDir):

    #join the path with the names of the images
    dataDirTemp = os.path.join(dataDir, str(img))
    images.append(dataDirTemp)
print("Images to be processed: ", images)

Images to be processed:  ['../images1/G000_IMG062.jpg', '../images1/G000_IMG087.jpg', '../images1/G000_IMG102.jpg', '../images1/G006_IMG048.jpg', '../images1/G006_IMG086.jpg', '../images1/G006_IMG119.jpg', '../images1/G019_IMG082.jpg', '../images1/G028_IMG015.jpg', '../images1/G028_IMG062.jpg', '../images1/G028_IMG098.jpg', '../images1/G028_IMG101.jpg', '../images1/G033_IMG043.jpg', '../images1/G033_IMG075.jpg', '../images1/G033_IMG088.jpg', '../images1/G033_IMG101.jpg', '../images1/G038_IMG074.jpg', '../images1/G038_IMG088.jpg', '../images1/G038_IMG103.jpg', '../images1/G038_IMG105.jpg', '../images1/G041_IMG042.jpg', '../images1/G041_IMG048.jpg', '../images1/G041_IMG088.jpg', '../images1/G041_IMG098.jpg', '../images1/G047_IMG053.jpg', '../images1/G047_IMG068.jpg', '../images1/G047_IMG102.jpg', '../images1/G047_IMG107.jpg', '../images1/G056_IMG017.jpg', '../images1/G056_IMG077.jpg', '../images1/G056_IMG097.jpg', '../images1/G058_IMG044.jpg', '../images1/G058_IMG074.jpg', '../images1/G0

In [ ]:
import cv2
import numpy as np
import copy

def findHourse(img_original, x_offset=0, y_offset=0, width_box=30, height_box=30):
    """
    Draws boxes at the four corners of an image.
    
    Parameters:
    - img: Input image (BGR format)
    - x_offset, y_offset: Offset for corner positions
    - width_box, height_box: Size of the corner boxes
    """
    img = img_original.copy()
    # Convert to grayscale (though we'll draw on the original color image)
    black_and_white = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    height, width = black_and_white.shape[:2]
    
    # Define the four corners with optional offsets
    corners = [
        (x_offset, height - 1 - height_box - y_offset),            # Bottom-left
        (y_offset, x_offset),                          # Top-left
        (width - 1 - height_box - y_offset, height - 1 - width_box - x_offset), # Bottom-right
        (width - 1 - width_box - x_offset , y_offset)             # Top-right
    ]
    convert = ["top_left",  "bottom_left","top_right", "bottom_right"]
    # Create a list to hold all box contours
    all_contours = []
    intensities = []
    count = 1
    for (x, y) in corners:
        if count%2 ==0:
            temp = width_box
            width_box = height_box
            height_box = temp
        count += 1
        # Define a rectangle (box) around the corner point
        box = np.array([
            [x, y],                          # Top-left of the box
            [x + width_box, y],               # Top-right of the box
            [x + width_box, y + height_box],  # Bottom-right of the box
            [x, y + height_box]               # Bottom-left of the box
        ], dtype=np.int32)
        
        all_contours.append(box)
        mask = np.zeros_like(black_and_white, dtype=np.uint8)
        cv2.drawContours(mask, [box], -1, 255, -1)  # Fill the contour (thickness=-1)
        mean_intensity = cv2.mean(black_and_white, mask=mask)[0]
        intensities.append(mean_intensity)
    index = np.argmin(intensities)

    # Draw all the box contours on the original image
    cv2.drawContours(
        img,
        [all_contours[index]],
        #all_contours,
        contourIdx=-1,          # Draw all contours
        color=(0, 255, 0),      # Green color (BGR)
        thickness=2             # Line thickness
    )
    oriented_board = copy.deepcopy(img)
    # Display results
    if index == 2:
        rotation_code = 90  
        oriented_board = cv2.rotate(oriented_board, cv2.ROTATE_90_CLOCKWISE)
    elif index == 3:
        rotation_code = 180  
        oriented_board = cv2.rotate(oriented_board, cv2.ROTATE_180)
    elif index == 1:
        rotation_code = 270 
        oriented_board = cv2.rotate(oriented_board, cv2.ROTATE_90_COUNTERCLOCKWISE)
    
    cv2.imshow("Image with Corner Boxes", img)
    cv2.imshow("Oriented Board", oriented_board)
    cv2.waitKey(0)
    #cv2.imshow("Grayscale Version", black_and_white)
    #cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return img

# Example usage:
# image = cv2.imread("your_image.jpg")
# processed_image = findCorners(image, width_box=40, height_box=40)

In [8]:
from task3_2API import detectBoardAndSquares
for img in images:#[:1]:
    print("Processing image: ", img)
    # Call the function to detect board and squares
    normalizedBoard, square_box, M, fx, fy = detectBoardAndSquares(img)
    if(normalizedBoard is not None):
        findHourse(normalizedBoard,10,5,45,70)
    print("Finished processing image: ", img)

Processing image:  ../images1/G000_IMG062.jpg
Chessboard found in ../images1/G000_IMG062.jpg
All squares found


UnboundLocalError: cannot access local variable 'oriented_board' where it is not associated with a value